<a href="https://colab.research.google.com/github/Pauloengcomp/StudyingPipeline/blob/main/TreinandoPipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np


from sklearn.pipeline import Pipeline, make_pipeline #pipeline mesmo
from sklearn.ensemble import RandomForestClassifier #xgbooster
from sklearn.linear_model import LogisticRegression #regressao linear/ regressao logistica
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder #scaling
from sklearn.compose import ColumnTransformer 
from sklearn.impute import KNNImputer, SimpleImputer #imputacao de dados faltantes  
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV

In [2]:
url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
train =  pd.read_csv(url)
test = pd.read_csv(url)

display(train.head())
display(test.head())

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
criacao_de_pipeline = Pipeline([
                                ('one_hot_encoder', OneHotEncoder(handle_unknown='ignore')),
                                ('min_max_scaler', StandardScaler(with_mean=False)),
                                ('classificador', RandomForestClassifier())
])

criacao_de_pipeline

Pipeline(steps=[('one_hot_encoder', OneHotEncoder(handle_unknown='ignore')),
                ('min_max_scaler', StandardScaler(with_mean=False)),
                ('classificador', RandomForestClassifier())])

In [4]:
make_pipeline(OneHotEncoder(handle_unknown='ignore'), StandardScaler(with_mean=False), RandomForestClassifier())

Pipeline(steps=[('onehotencoder', OneHotEncoder(handle_unknown='ignore')),
                ('standardscaler', StandardScaler(with_mean=False)),
                ('randomforestclassifier', RandomForestClassifier())])

In [9]:
x = train.drop('Survived', axis = 1) #variaveis que irão prever o Y
y = train['Survived']

x_train, x_test, y_train, y_test = train_test_split(x,y)

x_train.shape, x_test.shape, y_train.shape, y_test.shape

((668, 11), (223, 11), (668,), (223,))

In [10]:
criacao_de_pipeline.fit(x_train,y_train)

Pipeline(steps=[('one_hot_encoder', OneHotEncoder(handle_unknown='ignore')),
                ('min_max_scaler', StandardScaler(with_mean=False)),
                ('classificador', RandomForestClassifier())])

In [11]:
criacao_de_pipeline.predict(x_test)

array([0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0,
       1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,
       0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0,
       0, 0, 0])

In [12]:
criacao_de_pipeline.score(x_test,y_test)

0.8475336322869955

In [14]:
x_train.dtypes

PassengerId      int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [25]:
variaveis_categoricas = [coluna for  coluna in x_train.columns if x_train[coluna].dtype.name == 'object']
variaveis_categoricas

['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']

In [27]:
variaveis_numericas = [coluna for  coluna in x_train.columns if coluna not in variaveis_categoricas]
variaveis_numericas

['PassengerId', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare']

In [33]:
#tratar dados categoricos faltantes
pipeline_categoricos = Pipeline([
  ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
  ('enconder',OneHotEncoder(handle_unknown='ignore',sparse=False))
])

In [35]:
#tratar dados numericos
pipeline_numericos = Pipeline([
  ('imputer', SimpleImputer(strategy='median')),
  ('scaler',MinMaxScaler())
])

In [37]:
pre_processing = ColumnTransformer ([
  ('cat', pipeline_categoricos, variaveis_categoricas),
  ('num', pipeline_numericos, variaveis_numericas)
])

In [39]:
pipeline_random_florest = make_pipeline(pre_processing,RandomForestClassifier(random_state=42))
pipeline_log_reg = make_pipeline(pre_processing, LogisticRegression(random_state=42))

In [40]:
pipeline_random_florest

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('enconder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  ['Name', 'Sex', 'Ticket',
                                                   'Cabin', 'Embarked']),
                                                 ('num',
                                                  Pipeline(steps=[('imputer',
                                                   

In [41]:
#preprocessa e treina a base de treinamento
#faz a predicao do score na base de teste
pipeline_random_florest.fit(x_train, y_train)
pipeline_random_florest.score(x_test, y_test)

0.8430493273542601

In [42]:
pipeline_log_reg.fit(x_train, y_train)
pipeline_log_reg.score(x_test, y_test)

0.820627802690583

In [43]:
validacao_cruzada = KFold(n_splits=10, shuffle=True, random_state=42)

In [44]:
cross_val_score(pipeline_random_florest, x, y, cv=validacao_cruzada)

array([0.83333333, 0.79775281, 0.83146067, 0.7752809 , 0.87640449,
       0.87640449, 0.78651685, 0.79775281, 0.76404494, 0.93258427])

In [46]:
acuracia_media_rf = cross_val_score(pipeline_random_florest, x, y, cv=validacao_cruzada).mean()
acuracia_media_rf

0.8271535580524345

In [47]:
acuracia_media_rl = cross_val_score(pipeline_log_reg, x, y, cv=validacao_cruzada).mean()
acuracia_media_rl

0.8136579275905118